In [1]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output
  !pip install torchtext -qq
  !pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
  basepath = '/content/drive/MyDrive/hw6'
else:
  basepath = '/content/drive/MyDrive/hw6'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 413.6 kB/s eta 0:00:00


In [2]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb
from sklearn.model_selection import train_test_split

#Pandas
import pandas as pd

# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix,hamming_loss, f1_score
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

#For Preprocessor
from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import re
import spacy
import numpy as np
from nltk.stem.porter import PorterStemmer
import os
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/train.csv')

In [4]:
df = df.sample(frac=1)

In [5]:
df.head(2)

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
4225,2017-10469,oh these old things on my face? nah they're no...,1,0,0,0,0,0,1,0,0,0,0
595,2017-40122,@GCC_DiscGolf lost my xt nova around hole 8 or...,0,0,0,0,0,0,0,0,1,0,0


In [6]:
df.drop(['ID'],axis=1 , inplace = True)
y = df.drop(['Tweet'],axis=1).values.astype(float)
X = df['Tweet'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
trainset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})

validset = Dataset.from_dict({
    'text': X_val,
    'label': y_val
})

In [8]:
train_val = DatasetDict(
    {"train": trainset, "valid": validset})

In [9]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True, padding=True, return_tensors="pt")

In [11]:
tokenized_dataset= train_val.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['text']
)
tokenized_dataset.set_format(type='torch')

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 11)  # We are using the same checkpiont as we have used for tokenizer


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
config = AutoConfig.from_pretrained(checkpoint)
model.config = config

In [14]:
from sklearn.metrics import hamming_loss,f1_score

def compute_metrics(eval_pred):
    #combined_metrics = evaluate.combine([evaluate.load("accuracy"),
                                        #evaluate.load("f1", average="macro")])

    logits, labels = eval_pred
    labels = labels.astype('float')
    predictions = (logits>=0).astype('float')
    hamming_loss_value = hamming_loss(labels, predictions)
    f1 = f1_score(labels, predictions, average = 'macro')
    evaluations = {
        'hamming_loss' : hamming_loss_value,
        'f1_score': f1
    }
    return evaluations

In [15]:
# Define the directory where model checkpoints will be saved
run_name = "emotions_test_final"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=4,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    # auto_find_batch_size=True,
    weight_decay=0.01,  # Apply L2 regularization to prevent overfitting
    learning_rate=2e-5,  # Step size for the optimizer during training
    optim='adamw_torch',  # Optimizer,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=25,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=25,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="f1_score",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    fp16=True,


)


In [16]:
# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
 #specify the project name where the experiment will be logged
%env WANDB_PROJECT = bert_run_final

env: WANDB_PROJECT=bert_run_final


In [19]:
trainer.train()  # start training

wandb: Currently logged in as: ahsonriaz98 (ahson_ml). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Hamming Loss,F1 Score,Runtime,Samples Per Second,Steps Per Second
25,No log,0.513977,0.212180,0.000000,1.489300,1037.395000,16.786000
50,0.547100,0.468140,0.212180,0.000000,1.591300,970.913000,15.711000
75,0.547100,0.428820,0.163813,0.221487,1.020500,1513.903000,24.497000
100,0.439100,0.398171,0.160165,0.254616,2.158700,715.715000,11.581000
125,0.439100,0.383252,0.157046,0.257392,1.200100,1287.416000,20.832000
150,0.391400,0.368280,0.150162,0.290806,1.122400,1376.569000,22.275000
175,0.391400,0.356459,0.142571,0.317127,1.139700,1355.563000,21.935000
200,0.359100,0.349207,0.139747,0.342579,1.144000,1350.578000,21.854000
225,0.359100,0.343870,0.137393,0.358213,1.011400,1527.535000,24.717000
250,0.340700,0.339310,0.134098,0.369970,1.340700,1152.383000,18.647000


TrainOutput(global_step=388, training_loss=0.3820431748616327, metrics={'train_runtime': 182.7101, 'train_samples_per_second': 135.274, 'train_steps_per_second': 2.124, 'total_flos': 434906447156832.0, 'train_loss': 0.3820431748616327, 'epoch': 4.0})

In [20]:
eval_results = trainer.evaluate(tokenized_dataset["valid"])

In [21]:
eval_results

{'eval_loss': 0.328363835811615,
 'eval_hamming_loss': 0.13033245072080024,
 'eval_f1_score': 0.38740195622394824,
 'eval_runtime': 1.7974,
 'eval_samples_per_second': 859.563,
 'eval_steps_per_second': 13.909,
 'epoch': 4.0}

In [22]:
wandb.log({"eval_hamm": eval_results["eval_hamming_loss"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1_score"]})

In [23]:
valid_output = trainer.predict(tokenized_dataset["valid"])

In [24]:
valid_preds = (valid_output.predictions>0).astype(float)
valid_labels = valid_output.label_ids

In [25]:
print("Validation F1 Score: ", f1_score(valid_labels,valid_preds, average= 'macro'), "\nValidation Hamming Loss: ",hamming_loss(valid_labels,valid_preds))

Validation F1 Score:  0.38740195622394824 
Validation Hamming Loss:  0.13033245072080024


In [26]:
wandb.finish()

eval/f1_score,▁▁▅▆▆▆▇▇▇███████
eval/hamming_loss,██▄▄▃▃▂▂▂▁▁▁▁▁▁▁
eval/loss,█▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁
eval/runtime,▄▅▁█▂▂▂▂▁▃▂█▁▂▁▆
eval/samples_per_second,▄▃█▁▆▇▆▆█▅▇▁▇▆█▂
eval/steps_per_second,▄▃█▁▆▇▆▆█▅▇▁▇▆█▂
eval_f1,▁
eval_hamm,▁
eval_loss,▁
train/epoch,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/global_step,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████


In [27]:
test_df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/test.csv')

In [28]:
y_test = test_df.drop(['Tweet','ID'],axis=1)
y_test =y_test.replace('NONE',0)
y_test  = y_test.values.astype(float)

y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
testset = Dataset.from_dict({
    'text': test_df['Tweet'].values,
    'label': y_test
})

In [30]:
tokenized_test_dataset= testset.map(tokenize_fn, batched=True)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(
    ['text']
)
tokenized_test_dataset.set_format(type='torch')

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [31]:
tokenized_test_dataset

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3259
})

In [32]:
test_preds = trainer.predict(tokenized_test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [33]:
test_preds = (test_preds.predictions>0).astype('int32')

In [34]:
ID = test_df[['ID']]

In [35]:
predict_map_df = pd.DataFrame(test_preds, columns = test_df.drop(['Tweet','ID'],axis=1).columns)

In [36]:
bert_result = pd.concat([ID, predict_map_df],axis=1)

In [37]:
bert_result.to_csv('/content/drive/MyDrive/hw6/bert_experiment.csv')

In [38]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ROBERT

In [39]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  basepath = '/content/drive/MyDrive/hw6'
else:
  basepath = '/content/drive/MyDrive/hw6'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb
from sklearn.model_selection import train_test_split

#Pandas
import pandas as pd

# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix,hamming_loss, f1_score
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

#For Preprocessor
from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import re
import spacy
import numpy as np
from nltk.stem.porter import PorterStemmer
import os
from sklearn.model_selection import train_test_split

In [41]:
df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/train.csv')

In [42]:
df = df.sample(frac=1)

In [43]:
df.drop(['ID'],axis=1 , inplace = True)
y = df.drop(['Tweet'],axis=1).values.astype(float)
X = df['Tweet'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
trainset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})

validset = Dataset.from_dict({
    'text': X_val,
    'label': y_val
})

In [45]:
train_val = DatasetDict(
    {"train": trainset, "valid": validset})

In [46]:
checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, problem_type="multi_label_classification")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [47]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True, padding=True, return_tensors="pt")

In [48]:
tokenized_dataset= train_val.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['text']
)
tokenized_dataset.set_format(type='torch')

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

In [49]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 11)  # We are using the same checkpiont as we have used for tokenizer


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
config = AutoConfig.from_pretrained(checkpoint)
model.config = config

In [51]:
from sklearn.metrics import hamming_loss,f1_score

def compute_metrics(eval_pred):

    logits, labels = eval_pred
    labels = labels.astype('int32')
    predictions = (logits>=0).astype('int32')
    hamming_loss_value = hamming_loss(labels, predictions)
    f1 = f1_score(labels, predictions, average = 'macro')
    evaluations = {
        'hamming_loss' : hamming_loss_value,
        'f1_score': f1
    }
    return evaluations

In [52]:
# Define the directory where model checkpoints will be saved
run_name = "emotions_robert_final"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=2,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # auto_find_batch_size=True,
    weight_decay=0.01,  # Apply L2 regularization to prevent overfitting
    learning_rate=2e-5,  # Step size for the optimizer during training
    optim='adamw_torch',  # Optimizer,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=25,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=25,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="f1_score",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    fp16=True,


)


In [53]:
# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [54]:
wandb.login()

True

In [55]:
 #specify the project name where the experiment will be logged
%env WANDB_PROJECT = robert

env: WANDB_PROJECT=robert


In [56]:
trainer.train()  # start training

Step,Training Loss,Validation Loss,Hamming Loss,F1 Score,Runtime,Samples Per Second,Steps Per Second
25,No log,0.484308,0.213651,0.000000,1.444200,1069.804000,67.166000
50,0.538200,0.462544,0.213651,0.000000,1.659900,930.777000,58.437000
75,0.538200,0.426436,0.180877,0.168519,2.913600,530.269000,33.292000
100,0.434300,0.415211,0.186349,0.193010,2.874100,537.558000,33.750000
125,0.434300,0.396245,0.169873,0.246806,2.307700,669.504000,42.034000
150,0.401400,0.391049,0.172580,0.193791,1.610300,959.430000,60.236000
175,0.401400,0.389932,0.163283,0.285399,1.938200,797.123000,50.046000
200,0.396400,0.376881,0.155869,0.300086,1.399300,1104.137000,69.321000
225,0.396400,0.370596,0.152398,0.328487,1.917300,805.838000,50.593000
250,0.373300,0.365913,0.149456,0.341667,1.371100,1126.819000,70.745000


TrainOutput(global_step=774, training_loss=0.36980795675469924, metrics={'train_runtime': 438.7396, 'train_samples_per_second': 28.167, 'train_steps_per_second': 1.764, 'total_flos': 496397228930652.0, 'train_loss': 0.36980795675469924, 'epoch': 2.0})

In [57]:
eval_results = trainer.evaluate(tokenized_dataset["valid"])

In [58]:
eval_results

{'eval_loss': 0.3276537358760834,
 'eval_hamming_loss': 0.1325684024713151,
 'eval_f1_score': 0.4346359495678214,
 'eval_runtime': 1.4077,
 'eval_samples_per_second': 1097.503,
 'eval_steps_per_second': 68.905,
 'epoch': 2.0}

In [59]:
wandb.log({"eval_hamm": eval_results["eval_hamming_loss"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1_score"]})

In [60]:
valid_output = trainer.predict(tokenized_dataset["valid"])
valid_preds = (valid_output.predictions>0).astype(float)
valid_labels = valid_output.label_ids

In [61]:
wandb.finish()

eval/f1_score,▁▁▄▄▅▄▆▆▆▇▆▇▇▇▇▇▇▇▇██▇█████████
eval/hamming_loss,██▅▆▄▄▄▃▃▂▂▂▂▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁
eval/loss,█▇▅▅▄▄▄▃▃▃▃▂▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▁▃▃▂▁▂▁▂▁▁▁▂▂▂▂▅▅▄▂▃██▆▄▄▄▇▁▂▁
eval/samples_per_second,█▆▃▃▄▇▅█▅███▅▅▅▅▂▂▃▅▃▁▁▂▃▃▃▁█▅█
eval/steps_per_second,█▆▃▃▄▇▅█▅███▅▅▅▅▂▂▃▅▃▁▁▂▃▃▃▁█▅█
eval_f1,▁
eval_hamm,▁
eval_loss,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████


In [62]:
test_df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/test.csv')

In [63]:
y_test = test_df.drop(['Tweet','ID'],axis=1)
y_test =y_test.replace('NONE',0)
y_test  = y_test.values.astype(float)

y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
testset = Dataset.from_dict({
    'text': test_df['Tweet'].values,
    'label': y_test
})

In [65]:
tokenized_test_dataset= testset.map(tokenize_fn, batched=True)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(
    ['text']
)
tokenized_test_dataset.set_format(type='torch')

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [66]:
tokenized_test_dataset

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3259
})

In [67]:
test_preds = trainer.predict(tokenized_test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [68]:
test_preds = (test_preds.predictions>0).astype('int32')

In [69]:
ID = test_df[['ID']]

In [70]:
predict_map_df = pd.DataFrame(test_preds, columns = test_df.drop(['Tweet','ID'],axis=1).columns)

In [71]:
robert_result = pd.concat([ID, predict_map_df],axis=1)

In [72]:
robert_result.head(2)

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,0,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,0,0,0


In [73]:
robert_result.to_csv('/content/drive/MyDrive/hw6/robert_experiment.csv')

flan t5

In [77]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output

  basepath = '/content/drive/MyDrive/hw6'
else:
  basepath = '/content/drive/MyDrive/hw6'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb
from sklearn.model_selection import train_test_split

#Pandas
import pandas as pd

# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix,hamming_loss, f1_score
from datetime import datetime

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

#For Preprocessor
from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import re
import spacy
import numpy as np
from nltk.stem.porter import PorterStemmer
import os
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/train.csv')

In [5]:
df = df.sample(frac=1)

In [6]:
df.drop(['ID'],axis=1 , inplace = True)
y = df.drop(['Tweet'],axis=1).values.astype('float')
X = df['Tweet'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
trainset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})

validset = Dataset.from_dict({
    'text': X_val,
    'label': y_val
})

In [8]:
train_val = DatasetDict(
    {"train": trainset, "valid": validset})

In [9]:
checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True, padding=True, return_tensors="pt")

In [11]:
tokenized_dataset= train_val.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['text']
)
tokenized_dataset.set_format(type='torch')

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 6179
    })
    valid: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1545
    })
})

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 11)  # We are using the same checkpiont as we have used for tokenizer


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
config = AutoConfig.from_pretrained(checkpoint)
model.config = config

In [15]:
from sklearn.metrics import hamming_loss,f1_score

def compute_metrics(eval_pred):
    #combined_metrics = evaluate.combine([evaluate.load("accuracy"),
                                        #evaluate.load("f1", average="macro")])

    logits, labels = eval_pred
    labels = labels.astype('int32')

    if isinstance(
        logits, tuple
    ):  # if the model also returns hidden_states or attentions
        logits = logits[0]

    predictions = (logits>=0).astype('int32')
    hamming_loss_value = hamming_loss(labels, predictions)
    f1 = f1_score(labels, predictions, average = 'macro')
    evaluations = {
        'hamming_loss' : hamming_loss_value,
        'f1_score': f1
    }
    return evaluations

In [36]:
# Define the directory where model checkpoints will be saved
run_name = "emotions_flan_final"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=2,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    #gradient_accumulation_steps=8,

    weight_decay=0.1,  # Apply L2 regularization to prevent overfitting
    learning_rate=1e-4,  # Step size for the optimizer during training
    lr_scheduler_type='linear',
    warmup_steps=0,  # Number of warmup steps for the learning rate scheduler
    optim='adamw_torch',  # Optimizer,
    max_grad_norm = 1.0,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=150,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=150,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="f1_score",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=150,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    fp16=False,
    bf16=False,
    tf32= False
)


In [37]:
# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [38]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [39]:
 #specify the project name where the experiment will be logged
%env WANDB_PROJECT = flant5

env: WANDB_PROJECT=flant5


In [40]:
trainer.train()  # start training

Step,Training Loss,Validation Loss,Hamming Loss,F1 Score,Runtime,Samples Per Second,Steps Per Second
150,0.253600,0.313451,0.127332,0.512711,20.382100,75.802000,9.518000
300,0.223900,0.321381,0.125449,0.526117,19.026000,81.205000,10.197000
450,0.212000,0.332007,0.125802,0.525804,20.107600,76.837000,9.648000
600,0.211700,0.335022,0.126508,0.536095,18.695500,82.640000,10.377000
750,0.205800,0.331214,0.127979,0.537613,17.365600,88.969000,11.171000
900,0.253700,0.315120,0.124507,0.538889,17.371000,88.941000,11.168000
1050,0.257400,0.321299,0.126096,0.538406,18.880500,81.830000,10.275000
1200,0.263500,0.314848,0.126155,0.547702,18.059400,85.551000,10.742000
1350,0.260000,0.313546,0.124390,0.548577,19.604200,78.810000,9.896000
1500,0.257000,0.311630,0.123330,0.544668,17.456900,88.504000,11.113000


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=1546, training_loss=0.2406798574933722, metrics={'train_runtime': 1037.7463, 'train_samples_per_second': 11.908, 'train_steps_per_second': 1.49, 'total_flos': 1197711994475094.0, 'train_loss': 0.2406798574933722, 'epoch': 2.0})

In [41]:
eval_results = trainer.evaluate(tokenized_dataset["valid"])

In [42]:
eval_results

{'eval_loss': 0.31354567408561707,
 'eval_hamming_loss': 0.1243895263312739,
 'eval_f1_score': 0.5485766152180431,
 'eval_runtime': 18.0093,
 'eval_samples_per_second': 85.789,
 'eval_steps_per_second': 10.772,
 'epoch': 2.0}

In [43]:
wandb.log({"eval_hamm": eval_results["eval_hamming_loss"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1_score"]})

In [44]:
valid_output = trainer.predict(tokenized_dataset["valid"])

In [45]:
#logits_test, tup2 = valid_output
predictions = (valid_output[0][0]>=0).astype('int32')
valid_labels = valid_output.label_ids

In [46]:
valid_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [47]:
print("Validation F1 Score: ", f1_score(valid_labels,predictions, average= 'macro'), "\nValidation Hamming Loss: ",hamming_loss(valid_labels,predictions))

Validation F1 Score:  0.5485766152180431 
Validation Hamming Loss:  0.1243895263312739


In [48]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 1350.


In [49]:
wandb.finish()

eval/f1_score,▂▁▁▄▄▅▅▅▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
eval/hamming_loss,█▇▇▄▄▃▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,██▇▆▅▄▅▃▃▃▃▂▂▂▂▂▃▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▁▁
eval/runtime,▃█▆▃▁▂▂▄▃▁▁▂▃▃▂▃▃▃▃▁▁▁▂▁▁▂▁▁▁▂▁▃▃▂▁▁▂▁▂▁
eval/samples_per_second,▆▁▂▅▇▇▇▄▆█▇▇▆▅▆▆▆▆▅▇█▇▇▇█▇▇█▇▇▇▅▅▇██▇▇▆▇
eval/steps_per_second,█▄▅▁▂▂▂▁▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂
eval_f1,▁
eval_hamm,▁
eval_loss,▁
train/epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▂▂▃▄▄▅▅▆▆▇██
train/global_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▂▂▃▄▄▅▆▆▇███


In [50]:
test_df = pd.read_csv('/content/drive/MyDrive/hw5/emotion-detection-spring2014/test.csv')

In [51]:
y_test = test_df.drop(['Tweet','ID'],axis=1)
y_test =y_test.replace('NONE',0)
y_test  = y_test.values.astype(float)

y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
testset = Dataset.from_dict({
    'text': test_df['Tweet'].values,
    'label': y_test
})

In [53]:
tokenized_test_dataset= testset.map(tokenize_fn, batched=True)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(
    ['text']
)
tokenized_test_dataset.set_format(type='torch')

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [54]:
test_preds = trainer.predict(tokenized_test_dataset)

In [55]:
test_preds[0][0]

array([[ 0.22994536, -2.0275722 ,  0.34772757, ..., -3.2469962 ,
        -6.112753  , -4.87858   ],
       [ 1.1144378 , -3.3644133 ,  0.8940943 , ..., -0.34800693,
        -4.9315014 , -5.0696173 ],
       [ 2.1565797 , -3.5090954 ,  2.1755211 , ...,  1.592011  ,
        -4.0790825 , -7.6280336 ],
       ...,
       [ 6.1383996 , -4.5875783 ,  3.3349385 , ..., -3.030362  ,
        -4.668382  , -7.0656996 ],
       [-4.5553913 , -3.5779245 , -5.277474  , ..., -5.399752  ,
        -2.7902396 , -2.877236  ],
       [-2.4481962 , -2.292343  , -2.2748616 , ..., -4.5155826 ,
        -3.1343386 , -4.095445  ]], dtype=float32)

In [56]:
test_preds = (test_preds[0][0]>0).astype('int32')

In [57]:
test_preds[1]

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [58]:
ID = test_df[['ID']]

In [59]:
predict_map_df = pd.DataFrame(test_preds, columns = test_df.drop(['Tweet','ID'],axis=1).columns)

In [60]:
flan_result = pd.concat([ID, predict_map_df],axis=1)

In [61]:
flan_result.head(2)

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,0,0,0


In [62]:
flan_result.to_csv('/content/drive/MyDrive/hw6/flan_experiment.csv')